# **<center><font style="color:rgb(100,109,254)">Module 3: Advance Gesture Controlled Shape/Object Manipulation </font></center>**

<center>
    <img src='https://drive.google.com/uc?export=download&id=1OGxEgnz1eeMKP-y9dvYtBfV1Zc5VR1to'>
    <a href='https://www.microsoft.com/en-us/hololens/developers'>HoloLens photo courtesy of Microsoft</a>
</center>


## **<font style="color:rgb(134,19,348)"> Module Outline </font>**

The module can be split into the following parts:


- *Lesson 1: Create a Basic Hand Paint Application*

- ***Lesson 2:* Add Adjustable Paint Color Functionality** *(This Tutorial)*

- *Lesson 3: Draw Shapes/Objects utilizing Hand Gestures*

- *Lesson 4: Manipulate Shapes/Objects utilizing Hand Gestures*

**Please Note**, these Jupyter Notebooks are not for sharing; do read the Copyright message below the Code License Agreement section, which is in the last cell of this notebook.
-Taha Anwar

Alright, without further ado, let's dive in.

### **<font style="color:rgb(134,19,348)"> Import the Libraries</font>**

First, we will import the required libraries.

In [2]:
import cv2
import numpy as np
import mediapipe as mp
from collections import deque
from scipy.interpolate import interp1d
from previous_lesson import (detectHandsLandmarks, calculateDistance, recognizeGestures, draw)

## **<font style="color:rgb(134,19,348)">Initialize the Hands Landmarks Detection Model</font>**

After that, as we have been doing in the previous lessons, we will need to initialize the **`mp.solutions.hands`** class and then set up the **`mp.solutions.hands.Hands()`** function with appropriate arguments and also initialize **`mp.solutions.drawing_utils`** class that is needed to visualize the detected landmarks. 

In [3]:
# Initialize the mediapipe hands class.
mp_hands = mp.solutions.hands

# Set up the Hands functions for videos.
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, 
                       min_detection_confidence=0.8, min_tracking_confidence=0.8)

# Initialize the mediapipe drawing class.
mp_drawing = mp.solutions.drawing_utils

## **<font style="color:rgb(134,19,348)">Create a Function to Change Paint Color</font>**

Now we will create a function **`changePaintColor()`** that will change the color of our paintbrush utilizing hand gestures. As done in the first module, we will use the [**`scipy.interpolate.interp1d()`**](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html) function to increase/decrease the hue, saturation, and brightness values of the paint color in real-time with the increase/decrease of the distance between the fingers.

In [4]:
def changePaintColor(frame, paint_color, hand_gesture, distance):
    '''
    This function will change the paint brush color utilizing hand gestures.
    Args:
        frame:        The current frame/image (with hands in) of a real-time webcam feed.
        paint_color:  The blue, green, and red value of the current paint color.
        hand_gesture: The current hand gesture recognized in the frame.
        distance:     The distance between the middle finger and the thumb of a hand in the frame.
    Returns:
        frame         The same frame/image with the current active mode written on it with the paint color.
        new_color:    The new blue, green, and red value of the paint color.
    '''
    
    # Check if the current hand gesture is PINKY POINTING UP. 
    if hand_gesture == 'PINKY POINTING UP':
        
        # This means that the hue channel [0-179] is to be modified.
        # So get the interpolation function accordingly.
        color_interp_f = interp1d([30,230], [0, 179])
    
    # Otherwise.    
    else:
        
        # This means that the saturation or value channel [0-255] is to be modified.
        # So get the interpolation function accordingly.
        color_interp_f = interp1d([30,230], [0, 255])
        
    # Calculate the new paint color value.    
    new_color_value = color_interp_f(distance)
    
    # Convert the list into a 8-bit unsigned integer (0 to 255) array.
    # And also convert the color space of the paint color from BGR to HSV. 
    color_hsv = cv2.cvtColor(np.uint8([[paint_color]]), cv2.COLOR_BGR2HSV)
    
    # Convert the paint color into the float type.
    color_hsv = np.array(color_hsv, dtype=np.float64)
    
    # Split the hue, saturation, and value channel of the paint color.
    hue_channel, saturation_channel, value_channel = cv2.split(color_hsv)
    
    # Check if the current hand gesture is PINKY POINTING UP. 
    if hand_gesture == 'PINKY POINTING UP':
        
        # Write the current mode on the frame with the paint color.
        cv2.putText(img=frame, text='Change Paint Color Hue', org=(10, 30),fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1, color=paint_color, thickness=2)
        
        # Update the hue channel value of the paint color.
        hue_channel = np.array([[new_color_value]], dtype=np.float64) 
        
    # Check if the current hand gesture is 'MIDDLE RING PINKY POINTING UP'.
    elif hand_gesture == 'MIDDLE RING PINKY POINTING UP':
        
        # Write the current mode on the frame with the paint color.
        cv2.putText(img=frame, text='Change Paint Color Saturation', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1, color=paint_color, thickness=2)
        
        # Update the saturation channel value of the paint color.
        saturation_channel = np.array([[new_color_value]], dtype=np.float64) 
    
    # Check if the current hand gesture is ALL FINGERS POINTING UP.
    elif hand_gesture == 'ALL FINGERS POINTING UP':
        
        # Write the current mode on the frame with the paint color.
        cv2.putText(img=frame, text='Change Paint Color Brightness', org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                    fontScale=1, color=paint_color, thickness=2)
        

        # Update the value channel value of the paint color.
        value_channel = np.array([[new_color_value]], dtype=np.float64) 

    # Merge the Hue, Saturation, and Value channel.
    color_hsv = cv2.merge((hue_channel, saturation_channel, value_channel))
    
    # Clip (limit) the values between 0 and 255.
    # This will set values > 255 to 255 and values < 1 to 1.
    color_hsv = np.clip(a=color_hsv, a_min=1, a_max=255)
    
    # Convert the paint color into uint8 type and BGR color space.
    # Also flatten the array.
    color_bgr = cv2.cvtColor(np.array(color_hsv, dtype=np.uint8), cv2.COLOR_HSV2BGR).flatten()
    
    # Get the new color bgr values in a tuple.
    new_color = (int(color_bgr[0]), int(color_bgr[1]), int(color_bgr[2]))  
    
    # Return the frame along with the new paint color.
    return frame, new_color

## **<font style="color:rgb(134,19,348)">Create a Function to Visualize Paint Color Range</font>**

Now we have a function **`changePaintColor()`**  to change the paint color but without visualization of the possible options it is kind of impossible to choose the one we need. We can try out all the values and then choose the one we liked but that approach is very time-consuming. So now we will create a function  **`vizualizeColorRange()`** that will visualize the paint color range of the selected channel (i.e., Hue, Saturation, or Value) on the frame in real-time whenever we will make the gesture required to change the color.


<center>
    <img src='https://drive.google.com/uc?export=download&id=1eXTjdnMiA7AIRJjY62pzCt95LOabghXB' width=600>
</center>

In [5]:
def vizualizeColorRange(frame, paint_color_bgr, hand_gesture, distance, BAR_WIDTH=50, BAR_HEIGHT=256*2):
    '''
    This function will display a range bar of the paint color's hue, saturation, or value channel.
    Args:
        frame:           The frame/image on which the range bar visualization is required.
        paint_color_bgr: The blue, green, and red value of the current paint color.
        hand_gesture:    The current hand gesture recognized in the frame.
        distance:        The distance between the middle finger and the thumb of a hand in the frame.
        BAR_WIDTH:       A constant containing the height of the the range bar image.
        BAR_HEIGHT:      A constant containing the width of the the range bar image. This should be a multiple of 256.
    Returns:
        frame: The frame/image with the range bar of the paint color visualized.
    '''
    
    # Get the height and width of the frame of the webcam video.
    frame_height, frame_width, _ = frame.shape
     
    # Convert the list into a 8-bit unsigned integer (0 to 255) array.
    # And also convert the color space of the paint color from BGR to HSV. 
    paint_color_hsv = cv2.cvtColor(np.uint8([[paint_color_bgr]]), cv2.COLOR_BGR2HSV)
    
    # Convert the paint color into the float type.
    paint_color_hsv = np.array(paint_color_hsv, dtype=np.float64)
    
    # Split the hue, saturation, and value channel of the paint color.
    hue, saturation, value = cv2.split(paint_color_hsv)
    
    # Make a hue channel with all values equal to the hue of the paint color.
    hue_channel = hue*np.ones(shape=(int(BAR_HEIGHT/256),BAR_WIDTH), dtype=np.uint8)
    
    # Make a saturation channel with all values equal to the saturation of the paint color.
    saturation_channel = saturation*np.ones(shape=(int(BAR_HEIGHT/256),BAR_WIDTH), dtype=np.uint8)
    
    # Make a value channel with all values equal to the value (brightness) of the paint color.
    value_channel = value*np.ones(shape=(int(BAR_HEIGHT/256),BAR_WIDTH), dtype=np.uint8)
    
    # Initialize a list to store the range of all possible colors.
    colors=[]
        
    # Iterate over all possible channel values [0-255].
    for i in np.arange(0, 256):
        
        # Check if the current hand gesture is PINKY POINTING UP. 
        if hand_gesture == 'PINKY POINTING UP':
            
            # Check if the i is greater than 179.
            if i>179:
                
                # Break the loop.
                break

            # Make a hue channel with all values equal to i.
            hue_channel = i*np.ones((int(BAR_HEIGHT/256), BAR_WIDTH), dtype=np.float64)
            
        # Check if the current hand gesture is 'MIDDLE RING PINKY POINTING UP'.
        elif hand_gesture == 'MIDDLE RING PINKY POINTING UP':
            
            # Make a saturation channel with all values equal to i.
            saturation_channel = i*np.ones((int(BAR_HEIGHT/256), BAR_WIDTH), dtype=np.float64)
            
        # Check if the current hand gesture is ALL FINGERS POINTING UP.
        elif hand_gesture == 'ALL FINGERS POINTING UP':
            
            # Make a value channel with all values equal to i.
            value_channel = i*np.ones((int(BAR_HEIGHT/256), BAR_WIDTH), dtype=np.float64)
            
        # Merge the Hue, Saturation, and Value channel.
        # And also convert the color into a uint8 type array.
        color_hsv = np.array(cv2.merge((hue_channel, saturation_channel, value_channel)), dtype=np.uint8)

        # Append the color into the colors list.
        colors.append(color_hsv)
    
    # Now we will vertically stack all the colors in the range.
    ######################################################################################################################
    
    # Initialize the range image with the first color in the range list.
    # Also Convert it into BGR color space.
    resultant_image = cv2.cvtColor(colors[0], cv2.COLOR_HSV2BGR)
    
    # Iterate from [1 to the length of the colors list].
    # We are skipping 0 because we have already initialized the range image with the first color.
    # Now we have to iterate over the remaining colors in the range list.
    for i in range(1, len(colors)):
        
        # Convert the ith color in the list into BGR color space.
        color_bgr = cv2.cvtColor(colors[i], cv2.COLOR_HSV2BGR)
        
        # Vertically concatenate (stack) the color we are iterating upon into the range image.
        resultant_image = cv2.vconcat([color_bgr, resultant_image])
        
    ######################################################################################################################
    
    # Resize the resultant image according to the required size.
    # As hue channel has lower value range than the other channels so this is done to get a consistent size for the range images.
    resultant_image = cv2.resize(resultant_image, dsize=(BAR_WIDTH, BAR_HEIGHT))
    
    #cv2.imwrite('resultant_image.png', resultant_image)
    
    # Overlay the resultant image on the frame.
    frame[frame_height-600:(frame_height-600)+BAR_HEIGHT,
          frame_width-120:(frame_width-120)+BAR_WIDTH] = resultant_image
    
    # Draw a rectangle around the overlayed range image on the frame.
    cv2.rectangle(img=frame, pt1=(frame_width-120, frame_height-600),
                  pt2=((frame_width-120)+BAR_WIDTH, (frame_height-600)+BAR_HEIGHT),
                  color=(255, 255, 255), thickness=3)


    
    # Get the interpolation function and calculate the current color bar value.
    # This will be used to draw a arrow highlighting the current paint color in the overlayed color range image.
    bar_interp_f = interp1d([30,230],  [(frame_height-600)+BAR_HEIGHT, frame_height-600])
    bar_value = int(bar_interp_f(distance))

    # Get the contour points of the arrow we have to draw.
    pts = [(frame_width-150, bar_value+15), (frame_width-150, bar_value-15),
           (frame_width-120, bar_value)]

    # Draw the filled arrow highlighting the current paint color in the overlayed color range image.
    frame = cv2.drawContours(image=frame, contours=[np.array(pts, np.int32)], contourIdx=0, 
                     color=(255, 255, 255), thickness=-1)
    
    # Return the frame with the paint color range image overlayed.
    return frame

Now that we have the functions **`changePaintColor()`** and **`vizualizeColorRange()`** to add adjustable paint color functionality in our paint application from the previous lesson.  We will utilize a few functions from the first module like the **`recognizeGestures()`** to recognize the hand gestures and **`calculateDistance()`** to get the distance between the middle fingertip and thumb tip landmark of a hand. And then change the paint color based on this distance in rea;-time.

In [10]:
# Initialize the VideoCapture object to read from the webcam.
camera_video = cv2.VideoCapture(0, cv2.CAP_DSHOW)
camera_video.set(3,1280)
camera_video.set(4,960)


# Create named window for resizing purposes.
cv2.namedWindow('Hand Paint', cv2.WINDOW_NORMAL)

# Initialize variables to store previous x and y location.
# That are hand brush x and y coordinates in the previous frame.
prev_x = None 
prev_y = None

# Initialize a canvas to draw on.
canvas = np.zeros(shape=(int(camera_video.get(cv2.CAP_PROP_FRAME_HEIGHT)),
                         int(camera_video.get(cv2.CAP_PROP_FRAME_WIDTH)), 3),
                  dtype=np.uint8)

# Initialize a variable to store the color value.
paint_color = [0, 255, 0]

# Initialize a variable to store the hand label for gesture recognition.
hand_label = 'RIGHT'

# Initialize a variable to store the buffer length.
BUFFER_MAX_LENGTH = 2

# Initialize a buffer to store recognized gestures.
buffer = deque([], maxlen=BUFFER_MAX_LENGTH)

# Iterate until the webcam is accessed successfully.
while camera_video.isOpened():
   
    # Read a frame.
    ok, frame = camera_video.read()
    
    # Check if frame is not read properly then 
    # continue to the next iteration to read the next frame.
    if not ok:
        continue
    
    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)

    # Get the height and width of the frame of the webcam video.
    frame_height, frame_width, _ = frame.shape
    
    # Perform Hands landmarks detection on the frame.
    frame, results = detectHandsLandmarks(frame, hands, draw=True, display=False)
    
    # Check if the hands landmarks in the frame are detected.
    if results.multi_hand_landmarks:
        
        # Perform a hand gesture recognition.
        # I have modified this recognizeGestures() function,
        # to return the fingers tips position of the both hands.
        current_gesture, hands_tips_positions = recognizeGestures(frame, results,
                                                                  hand_label, draw=False,
                                                                  display=False)
        # Check if a known gesture is recognized.
        if current_gesture != 'UNKNOWN':
            
            # Check if all the gestures stored in the buffer are equal to the current gesture.
            if all(current_gesture==gesture for gesture in buffer):
                
                # Append the current gesture into the buffer.
                buffer.append(current_gesture)
                
            # Otherwise.
            else:
                
                # Clear the buffer.
                buffer.clear()
            
            # Check if the length of the buffer is equal to the maxlength, that is 10.
            if len(buffer) == BUFFER_MAX_LENGTH:
                
                # Draw, Erase or Clear the canvas depending upon the current gesture.
                canvas, (prev_x, prev_y) = draw(frame, canvas, current_gesture,
                                                hands_tips_positions[hand_label],
                                                (prev_x, prev_y), paint_color)
                
                # Check if the current hand gesture is 'PINKY POINTING UP', 'MIDDLE RING PINKY POINTING UP', or 'ALL FINGERS POINTING UP'.
                if current_gesture == 'PINKY POINTING UP' or current_gesture == 'MIDDLE RING PINKY POINTING UP' or \
                current_gesture == 'ALL FINGERS POINTING UP':
                
                    # Calculate the distance between the middle finger tip and thumb tip landmark of the other hand.
                    distance = calculateDistance(frame, hands_tips_positions['LEFT' if hand_label == 'RIGHT' else 'RIGHT']['MIDDLE'],
                                                 hands_tips_positions['LEFT' if hand_label == 'RIGHT' else 'RIGHT']['THUMB'], display=False)

                    # Check if the distance is calculated successfully.
                    # This will be none in case when the hand is not in the frame.
                    if distance:
                        
                        # Write the current distance percentage on the frame.
                        cv2.putText(img=frame, text=f'{int((distance-30)/2)}%',org=(frame_width-130, frame_height-630), 
                                    fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=3, color=paint_color, thickness=3)
                        
                        
                        # Change the paint color utilizing hand gestures.
                        frame, paint_color = changePaintColor(frame, paint_color, current_gesture, distance)
                        
                        # Display the paint color range.
                        frame = vizualizeColorRange(frame, paint_color, current_gesture, distance)
                        
                    
            # Otherwise.
            else:

                # Reset, by updating the previous x and y values to None.
                # This is required to start a new drawing.
                prev_x = None
                prev_y = None
               
    # Otherwise.
    else:
        
        # Clear the buffer.
        buffer.clear()

    # Update the pixel values of the frame with the canvas's values at the indexes where canvas!=0
    # i.e. where canvas is not black and something is drawn there.
    # In short, this will copy the drawings from canvas to the frame.
    frame[np.mean(canvas, axis=2)!=0] = canvas[np.mean(canvas, axis=2)!=0]
    
    # Display the frame.
    cv2.imshow("Hand Paint", frame)
    
   # Wait for 1ms. If a key is pressed, retreive the ASCII code of the key.
    k = cv2.waitKey(1) & 0xFF
    
    # Check if 'ESC' is pressed and break the loop.
    if k == 27:
        break
    
    # Check if 's' key is pressed and switch the hand label.
    elif k == ord('s'):
        
        # Set gesture hand label to 'LEFT', if it was 'RIGHT',
        # Otherwise if it was 'LEFT', set it to 'RIGHT'.
        hand_label = 'LEFT' if hand_label == 'RIGHT' else 'RIGHT'
        
# Release the VideoCapture Object and close the windows.
camera_video.release()
cv2.destroyAllWindows()




# Additional comments:
#       - Since my device has low specs, I brought down the buffer size
#         it helped a lot in detection, and the program was able to 
#         recognize my gestures.
#       - However, I think for better computers that has higher FPS, it
#         is recommended to bring the buffer size back to 20 or even a 
#         little higher.
#       - Apart from that, I encountered a problem with this program 
#         while testing it.
#       - If i remove these: 
#               camera_video.set(3,1280)
#               camera_video.set(4,960)
#         I get an error with the shape 

Great! paint colors are changing as we intended.

### **<font style="color:rgb(255,140,0)"> Code License Agreement </font>**
```
Copyright (c) 2022 Bleedai.com

Feel free to use this code for your own projects commercial or noncommercial, these projects can be Research-based, just for fun, for-profit, or even Education with the exception that you’re not going to use it for developing a course, book, guide, or any other educational products.

Under *NO CONDITION OR CIRCUMSTANCE* you may use this code for your own paid educational or self-promotional ventures without written consent from Taha Anwar (BleedAI.com).

```
